In [1]:
%matplotlib widget

In [2]:
import numpy as np
import scipy.optimize as sopt
import matplotlib.pyplot as plt
#import pydae.ssa as ssa
#import pydae.grid_tools as gt
import time
import json
plt.style.use('presentation.mplstyle')

In [3]:
from grid_2bus4wire import grid_2bus4wire_class

In [4]:
def get_flow(grid_obj,bus_j,bus_k,mode='total',model='pydgrid_pydae'):
    if model == 'pydgrid_pydae':
        v_a   = syst.get_values(f'v_{bus_j}_a_r') + 1j* syst.get_values(f'v_{bus_j}_a_i')
        i_l_a = syst.get_values(f'i_l_{bus_j}_{bus_k}_a_r') + 1j* syst.get_values(f'i_l_{bus_j}_{bus_k}_a_i')
        v_b   = syst.get_values(f'v_{bus_j}_b_r') + 1j* syst.get_values(f'v_{bus_j}_b_i')
        i_l_b = syst.get_values(f'i_l_{bus_j}_{bus_k}_b_r') + 1j* syst.get_values(f'i_l_{bus_j}_{bus_k}_b_i')
        v_c   = syst.get_values(f'v_{bus_j}_c_r') + 1j* syst.get_values(f'v_{bus_j}_c_i')
        i_l_c = syst.get_values(f'i_l_{bus_j}_{bus_k}_c_r') + 1j* syst.get_values(f'i_l_{bus_j}_{bus_k}_c_i')
        s_a = v_a*np.conj(i_l_a)
        s_b = v_b*np.conj(i_l_b)
        s_c = v_c*np.conj(i_l_c)

        if mode == 'total':
            s_t = s_a + s_b + s_c
            return s_t
        if mode == 'abc':
            return s_a,s_b,s_c
        
def set_voltage(grid_obj,bus_name,voltage,phase):
    '''
    Set new power to a grid feeder.

    Parameters
    ----------
    grid_obj : object of pydgrid.grid class
    bus_name : string
        name of the grid feeder bus.
    voltage : real escalar
        phase-phase RMS voltage magnitude
    phase : real escalar.
        phase angle in degree.

    Returns
    -------
    None.

    '''

 
    v_a = voltage/np.sqrt(3)*np.exp(1j*np.deg2rad(phase))
    v_b = voltage/np.sqrt(3)*np.exp(1j*np.deg2rad(phase-240))
    v_c = voltage/np.sqrt(3)*np.exp(1j*np.deg2rad(phase-120))
    grid_obj.set_value(f'v_{bus_name}_a_r',v_a.real)
    grid_obj.set_value(f'v_{bus_name}_a_i',v_a.imag)
    grid_obj.set_value(f'v_{bus_name}_b_r',v_b.real)
    grid_obj.set_value(f'v_{bus_name}_b_i',v_b.imag)
    grid_obj.set_value(f'v_{bus_name}_c_r',v_c.real)
    grid_obj.set_value(f'v_{bus_name}_c_i',v_c.imag)
    
def phasor2inst(grid_obj,bus_name,magnitude='v',to_bus='',phases=['a','b','c'],Freq = 50,Dt=1e-4):
    
    omega = 2*np.pi*Freq
    out = []
    
    if magnitude == 'v':
        for ph in phases:
            Times = np.arange(0.0,grid_obj.T[-1,0],Dt)
            R = grid_obj.get_values(f'{magnitude}_{bus_name}_{ph}_r') 
            I = grid_obj.get_values(f'{magnitude}_{bus_name}_{ph}_i')
            R_ = np.interp(Times,grid_obj.T[:,0],R)
            I_ = np.interp(Times,grid_obj.T[:,0],I)
            R_I = R_ + 1j*I_
            cplx = np.sqrt(2)*np.exp(1j*omega*Times)*R_I
            out += [cplx.real]

    if magnitude == 'iline':
        for ph in phases:
            Times = np.arange(0.0,grid_obj.T[-1,0],Dt)
            R = grid_obj.get_values(f'i_l_{bus_name}_{to_bus}_{ph}_r') 
            I = grid_obj.get_values(f'i_l_{bus_name}_{to_bus}_{ph}_i')
            R_ = np.interp(Times,grid_obj.T[:,0],R)
            I_ = np.interp(Times,grid_obj.T[:,0],I)
            R_I = R_ + 1j*I_
            cplx = np.sqrt(2)*np.exp(1j*omega*Times)*R_I
            out += [cplx.real]

    return Times,out

In [5]:
syst = grid_2bus4wire_class()
syst.Dt = 1e-3
syst.decimation =1 
syst.update()

In [6]:
syst.initialization_tol = 1e-8

set_voltage(syst,'B1',400,0.0)
set_voltage(syst,'B2',400,0.0)

syst.initialize([{}],xy0=200)
syst.run([{'t_end':0.04}])
set_voltage(syst,'B1',405,0.0)
set_voltage(syst,'B2',400,0.0)
syst.run([{'t_end':0.08}])
syst.post();


In [7]:
plt.close('all')
fig, axes = plt.subplots(nrows=1,ncols=2, figsize=(8, 3), frameon=False, dpi=100, squeeze=False)

axes[0,0].plot(syst.T, syst.get_values('v_B1_a_r'), label="$V_{B1}$")

fig.tight_layout()
fig.savefig('sim.svg')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
plt.close('all')
fig, axes = plt.subplots(nrows=2,ncols=2, figsize=(9, 4), frameon=False, dpi=100, squeeze=False)

Times,[v_1_a,v_1_b,v_1_c] = phasor2inst(syst,'B1')
axes[0,0].plot(Times, v_1_a, label="$V_{B1}$")
axes[0,0].plot(Times, v_1_b, label="$V_{B1}$")
axes[0,0].plot(Times, v_1_c, label="$V_{B1}$")

Times,[v_2_a,v_2_b,v_2_c] = phasor2inst(syst,'B2')
axes[0,0].plot(Times, v_2_a, label="$V_{B2}$")
axes[0,0].plot(Times, v_2_b, label="$V_{B2}$")
axes[0,0].plot(Times, v_2_c, label="$V_{B2}$")

Times,[v_1_a,v_1_b,v_1_c] = phasor2inst(syst,'B1')
axes[1,0].plot(Times, v_1_a, label="$V_{B1}$")
axes[1,0].plot(Times, v_1_b, label="$V_{B1}$")
axes[1,0].plot(Times, v_1_c, label="$V_{B1}$")

Times,[v_2_a,v_2_b,v_2_c] = phasor2inst(syst,'B2')
axes[1,0].plot(Times, v_2_a, label="$V_{B2}$")
axes[1,0].plot(Times, v_2_b, label="$V_{B2}$")
axes[1,0].plot(Times, v_2_c, label="$V_{B2}$")

Times,[i_a,i_b,i_c] = phasor2inst(syst,'B1',magnitude='iline',to_bus='B2')
axes[0,1].plot(Times, i_a, label="$V_{B1}$")
axes[0,1].plot(Times, i_b, label="$V_{B1}$")
axes[0,1].plot(Times, i_c, label="$V_{B1}$")

p_a = v_1_a*i_a
p_b = v_1_b*i_b
p_c = v_1_c*i_c
p_t = p_a + p_b + p_c

axes[1,1].plot(Times, p_a, label="$V_{B1}$")
axes[1,1].plot(Times, p_b, label="$V_{B1}$")
axes[1,1].plot(Times, p_c, label="$V_{B1}$")
axes[1,1].plot(Times, p_t, label="$V_{B1}$")

s_t = get_flow(syst,'B1','B2')
axes[1,1].plot(syst.T, s_t.real, label="$V_{B1}$")
axes[1,1].plot(syst.T, s_t.imag, label="$V_{B1}$")


axes[1,0].set_ylim([315,335])
fig.tight_layout()
fig.savefig('sim.svg')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
r_km = 0.167 
x_km =   0.08
Len = 0.2
R_12 = Len*r_km 
X_12 = Len*x_km
Z_12 = R_12 + 1j*X_12
V_1 = 405/np.sqrt(3)
V_2 = 400/np.sqrt(3)

I_12 = (V_1-V_2)/Z_12

S_2 = 3*V_2*np.conj(I_12)
S_2

(48703.66589868445+23331.09743649554j)

In [50]:
s_t.imag

array([ 8.94894913e-11,  1.41999192e-10, -1.55299108e-11,  3.69797902e-11,
        8.94894913e-11,  1.41999192e-10, -1.55299108e-11,  3.69797902e-11,
        8.94894913e-11,  1.41999192e-10, -1.55299108e-11,  3.69797902e-11,
        8.94894913e-11,  1.41999192e-10, -1.55299108e-11,  3.69797902e-11,
        8.94894913e-11,  1.41999192e-10, -1.55299108e-11,  3.69797902e-11,
        8.94894913e-11,  1.41999192e-10, -1.55299108e-11,  3.69797902e-11,
        8.94894913e-11,  1.41999192e-10, -1.55299108e-11,  3.69797902e-11,
        8.94894913e-11,  1.41999192e-10, -1.55299108e-11,  3.69797902e-11,
        8.94894913e-11,  1.41999192e-10, -1.55299108e-11,  3.69797902e-11,
        8.94894913e-11,  1.41999192e-10, -1.55299108e-11,  3.69797902e-11,
        8.94894913e-11,  2.36227362e+04,  2.36227362e+04,  2.36227362e+04,
        2.36227362e+04,  2.36227362e+04,  2.36227362e+04,  2.36227362e+04,
        2.36227362e+04,  2.36227362e+04,  2.36227362e+04,  2.36227362e+04,
        2.36227362e+04,  

In [19]:
def get_voltage(grid_obj,bus_name):
    '''
    Get voltage module of a bus.

    Parameters
    ----------
    grid_obj : object of pydae class
    bus_name : string
        name of the bus.

    Returns
    -------
    phase-ground voltage module (V).

    '''
    v_a = syst.get_value(f'v_{bus_name}_a_r') + 1j* syst.get_value(f'v_{bus_name}_a_i')
    U_meas = np.abs(v_a) 
    
    return U_meas



array([4.16122707e-12+8.94894913e-11j, 4.16122707e-12+1.41999192e-10j,
       4.16122707e-12-1.55299108e-11j, 4.16122707e-12+3.69797902e-11j,
       4.16122707e-12+8.94894913e-11j, 4.16122707e-12+1.41999192e-10j,
       4.16122707e-12-1.55299108e-11j, 4.16122707e-12+3.69797902e-11j,
       4.16122707e-12+8.94894913e-11j, 4.16122707e-12+1.41999192e-10j,
       4.16122707e-12-1.55299108e-11j, 4.16122707e-12+3.69797902e-11j,
       4.16122707e-12+8.94894913e-11j, 4.16122707e-12+1.41999192e-10j,
       4.16122707e-12-1.55299108e-11j, 4.16122707e-12+3.69797902e-11j,
       4.16122707e-12+8.94894913e-11j, 4.16122707e-12+1.41999192e-10j,
       4.16122707e-12-1.55299108e-11j, 4.16122707e-12+3.69797902e-11j,
       4.16122707e-12+8.94894913e-11j, 4.16122707e-12+1.41999192e-10j,
       4.16122707e-12-1.55299108e-11j, 4.16122707e-12+3.69797902e-11j,
       4.16122707e-12+8.94894913e-11j, 4.16122707e-12+1.41999192e-10j,
       4.16122707e-12-1.55299108e-11j, 4.16122707e-12+3.69797902e-11j,
      